In [ ]:
import numpy as np
import pandas as pd

In [ ]:
mem = pd.DataFrame(np.arange(1024), columns=['palavra'], dtype='int64')

In [ ]:
cache = pd.DataFrame(data=np.zeros((64, 7)), index=np.arange(64), columns=['v', 'tag', 'sujo', 'p0', 'p1', 'p2', 'p3'], dtype='int64')

In [ ]:
def address(n):
    b = '{:032b}'.format(n)
    word_offset = int(b[-2:], 2)
    block_offset = int(b[-4:-2], 2)
    index = int(b[-10:-4], 2)
    tag = int(b[:-10], 2)

    return tag, index, block_offset, word_offset

In [ ]:
def search(n):
    tag, index, boff, woff = address(n)

    # primeira palavra da memória
    mem_addr = index * 2**2 + tag * 2**6

    # palavras da memória para o cache
    cache.iloc[index] = [1, tag, 0, mem.iloc[mem_addr+0]['palavra'],
                                    mem.iloc[mem_addr+1]['palavra'],
                                    mem.iloc[mem_addr+2]['palavra'],
                                    mem.iloc[mem_addr+3]['palavra']]


In [ ]:
def write(n, data):
    tag, index, block_offset, word_offset = address(n)

    if (cache['v'][index] == 0):
        search(n)

    if (cache['sujo'][index] == 1):
        write_back(index)

    cache['p'+str(block_offset)][index] = data
    cache['sujo'][index] = 1

In [ ]:
def read(n):
    tag, index, boff, woff = address(n)
    bloco = cache.iloc[index]
    word = 'p' + str(boff)
    hit = False;

    # hit
    if (bloco['v'] == 1) and (bloco['tag'] == tag):
        hit = True;
        return bloco[word], hit

    if (bloco['v'] == 1) and (bloco['tag'] != tag):

        # busca na memória
        if (bloco['sujo'] == 0):
            search(n)
            bloco = cache.iloc[index]
            return bloco[word], hit

        # bloco sujo
        else:
            write_back(index)
            search(n)
            bloco = cache.iloc[index]
            return bloco[word], hit
   # busca na memória
    search(n)
    bloco = cache.iloc[index]

    return bloco[word], hit

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test1.txt to test1 (2).txt


In [ ]:
def main():
    reads, writes, hits, misses, hit_rate, miss_rate, acesso = 0, 0, 0, 0, 0, 0, 0
    saida = []
    f = open('test1.txt', 'r')
    for line in f:
        line = line.strip()
        saida.append(line)
        line = line.split(' ')
        n = int(line[0])
        if (len(line) == 3):
            data = int(line[2], 2)
            write(n, data)
            writes += 1
            saida.append('W')
        else:
            w, hit = read(n)
            reads += 1
            acesso += 1
            if (hit):
                hits += 1
                saida.append('H')
            else:
                misses += 1
                saida.append('M')

    hit_rate = hits/acesso
    miss_rate = misses/acesso

    print('READS: ', reads)
    print('WRITES: ', writes)
    print('HITS: ', hits)
    print('MISSES: ', misses)
    print('HIT RATE: ', hit_rate)
    print('MISS RATE: ', miss_rate)
    print()

    for i in range(0, len(saida), 2):
        print(saida[i], saida[i+1])

main()

READS:  2
WRITES:  2
HITS:  1
MISSES:  1
HIT RATE:  0.5
MISS RATE:  0.5

5 1 00000000000000000000000000000101 W
5 0 H
12 1 00000000000000000000000000010010 W
25 0 M
